In [58]:
from Utils.my_utils import *
from Recommenders.GraphBased.P3alphaRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.NonPersonalizedRecommender import *
import os
from random import randint, seed

In [59]:
# setta anche il seed di random per sicurezza
seed = randint(1,1000)
print("Seed: " +str(seed))
np.random.seed(seed)
users, items, users_to_eval = readData()
URM, user_originalID_to_index, item_originalID_to_index, item_index_to_originalID = preProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

Seed: 808


In [60]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2633 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2170 (17.2%) Users that have less than 1 test interactions


In [61]:
output_folder_path = "result_experiments/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

saved_folder_path = "result_best_models/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
    
recommender_class = P3alphaRecommender    
n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [62]:
runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_full,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchBayesianSkopt: Resuming 'P3alphaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 519, 'alpha': 1.4397329326212795, 'normalize_similarity': True}
P3alphaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 340 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 1716.21 column/sec. Elapsed time 12.95 sec
EvaluatorHoldout: Processed 10005 (100.0%) in 11.16 sec. Users per second: 896
SearchBayesianSkopt: New best config found. Config 0: {'topK': 519, 'alpha': 1.4397329326212795, 'normalize_similarity': True} - results: PRECISION: 0.0403798, PRECISION_RECALL_MIN_DEN: 0.0834731, RECALL: 0.0756946, MAP: 0.0160468, MAP_MIN_DEN: 0.0341683, MRR: 0.1169491, NDCG: 0.0666899, F1: 0.0526651, HIT_RATE: 0.2967516, ARHR_ALL_HITS: 0.1366053, NOVELTY: 0.0052912, AVERAGE_POPULARITY: 0.1954041, DIVERSITY_MEAN_INTER_LI

In [63]:
# load the model with all best parameters
recommender_object = P3alphaRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip")

P3alphaRecommender: URM Detected 230 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.
P3alphaRecommender: Loading model from file 'result_experiments/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


In [64]:
# load the TopPop model with all best parameters
recommender_object_top_pop = TopPop(URM_train_full)
recommender_object_top_pop.load_model(saved_folder_path, file_name = recommender_object_top_pop.RECOMMENDER_NAME + "_my_own_save.zip")

TopPopRecommender: URM Detected 230 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 119 ( 0.5%) items with no interactions.
TopPopRecommender: Loading model from file 'result_best_models/TopPopRecommender_my_own_save.zip'
TopPopRecommender: Loading complete


In [65]:
# recommend items to users
rec_items_ind = []
for user_id in users_to_eval:
    if(user_id in users):
        # recommend users for which we already have some ratings
        rec_items_ind.append(recommender_object.recommend(user_originalID_to_index[user_id], cutoff=10))
    else:
        # recommend users with no stored rating
        rec_items_ind.append(recommender_object_top_pop.recommend(user_id, cutoff=10, remove_seen_flag=False))

In [66]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

MAP = 0.0927


In [67]:
storeRecommendations(users_to_eval, rec_items_ind, item_index_to_originalID)

In [69]:
# save the model with all best parameters overwriting the old one
# recommender_object.save_model(saved_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")

P3alphaRecommender: Saving model in file 'result_best_models/P3alphaRecommender_my_own_save.zip'
P3alphaRecommender: Saving complete
